<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Lab: Fun with Neural Nets

---

Below is a procedure for building a neural network to recognize handwritten digits.  The data is from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data), and you will submit your results to Kaggle to test how well you did!

1. Load the training data (`train.csv`) from [Kaggle](https://www.kaggle.com/c/digit-recognizer/data)
2. Setup X and y (feature matrix and target vector)
3. Split X and y into train and test subsets.
4. Preprocess your data

   - When dealing with image data, you need to normalize your `X` by dividing each value by the max value of a pixel (255).
   - Since this is a multiclass classification problem, keras needs `y` to be a one-hot encoded matrix
   
5. Create your network.
   - Remember that for multi-class classification you need a softmax activation function on the output layer.
   - You may want to consider using regularization or dropout to improve performance.
   
6. Train your network.
7. If you are unhappy with your model performance, try to tighten up your model by adding hidden layers, adding hidden layer units, chaining the activation functions on the hidden layers, etc.
8. Load in [Kaggle's](https://www.kaggle.com/c/digit-recognizer/data) `test.csv`
9. Create your predictions (these should be numbers in the range 0-9).
10. Save your predictions and submit them to Kaggle.

---

For this lab, you should complete the above sequence of steps for **_at least_** two of the four "configurations":

1. Using a `tensorflow` network (we did _not_ cover this in class!)
2. Using a `keras` convolutional network
3. Using a `keras` network with regularization
4. Using a `tensorflow` convolutional network (we did _not_ cover this in class!)

In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.utils import np_utils
from sklearn.model_selection import train_test_split


In [4]:
train = pd.read_csv('train.csv')
train.head(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = train[train.columns[1:]].values
y = train['label']

In [6]:
X = X / 255.
y = np_utils.to_categorical(y)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2003)

In [8]:
model = Sequential()
model.add(Dense(X_train.shape[1], input_shape=(784,), activation='relu'))
model.add(Dropout(.5))
model.add(Dense(y_train.shape[1], activation='softmax'))

2022-10-30 22:40:48.515148: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [10]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=100)

Epoch 1/5
315/315 [==============================] - 2s 5ms/step - loss: 0.3673 - accuracy: 0.8910 - val_loss: 0.1867 - val_accuracy: 0.9470
Epoch 2/5
315/315 [==============================] - 2s 5ms/step - loss: 0.1735 - accuracy: 0.9478 - val_loss: 0.1364 - val_accuracy: 0.9592
Epoch 3/5
315/315 [==============================] - 2s 5ms/step - loss: 0.1303 - accuracy: 0.9621 - val_loss: 0.1118 - val_accuracy: 0.9664
Epoch 4/5
315/315 [==============================] - 2s 5ms/step - loss: 0.1018 - accuracy: 0.9679 - val_loss: 0.0989 - val_accuracy: 0.9695
Epoch 5/5
315/315 [==============================] - 1s 5ms/step - loss: 0.0823 - accuracy: 0.9753 - val_loss: 0.0886 - val_accuracy: 0.9727


In [11]:
test = pd.read_csv('test.csv')
test = test / 255.

In [21]:
#pred = model.predict_classes(test.values)
predictions = (model.predict(X_test) > 0.5).astype("int32")
predictions

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int32)

In [22]:
test['Label'] = predictions
test['ImageId'] = range(1,test.shape[0] + 1)

ValueError: Length of values (10500) does not match length of index (28000)

In [20]:
test[['ImageId', 'Label']].to_csv('submission.csv', index=False)

KeyError: "None of [Index(['ImageId', 'Label'], dtype='object')] are in the [columns]"